In [11]:
## fill nas with median
## very rude transformation of dates
## test/train split
import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype
import xgboost as xgb
import matplotlib
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import missingno as msno
import pickle
from sklearn import preprocessing
from sklearn.metrics import roc_curve
from scipy import stats
from scipy.stats import zscore
# from fancyimpute import *

%matplotlib inline

In [15]:
def save_obj(obj, name ):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)
    
def display_all(df):
    with pd.option_context("display.max_rows", 1000):
        with pd.option_context("display.max_columns", 1000):
            display(df)

def batch_save(train_x, train_y, valid_x, valid_y, test, postfix):
    train_x.reset_index().to_feather("tmp/train_x_{}".format(postfix))
    train_y.reset_index().to_feather("tmp/train_y_{}".format(postfix))
    valid_x.reset_index().to_feather("tmp/valid_x_{}".format(postfix))
    valid_y.reset_index().to_feather("tmp/valid_y_{}".format(postfix))
    test.reset_index().to_feather("tmp/test_{}".format(postfix))
    
def batch_load(postfix):
    train_x = pd.read_feather("tmp/train_x_{}".format(postfix))
    train_y = pd.read_feather("tmp/train_y_{}".format(postfix))
    valid_x = pd.read_feather("tmp/valid_x_{}".format(postfix))
    valid_y = pd.read_feather("tmp/valid_y_{}".format(postfix))
    return train_x, train_y, valid_x, valid_y

def my_roc(y_true, y_prob):
    if isinstance(y_true,pd.core.series.Series):
        y_true = np.array(y_true.tolist())
    if isinstance(y_true,list):
        y_true = np.array(y_true)
    sort_index = np.argsort(y_prob)[::-1]
    y_prob = y_prob[sort_index]
    y_true = y_true[sort_index]
    num_p = y_true.sum()
    num_n = len(y_true) - num_p
    fp = 0
    tp = 0
    fps = []
    tps = []
    prob_prev = -99
    i = 0
    while i < len(y_true):
        if y_prob[i]!=prob_prev:
            fps.append(fp/num_n)
            tps.append(tp/num_p)
            prob_prev=y_prob[i]
        if y_true[i]==1:
            tp+=1
        else:
            fp+=1
        i+=1
    fps.append(fp/num_n)
    tps.append(tp/num_p)
    return np.array(fps), np.array(tps)

def my_score3(predictions, xtrain): ##Adapted from SKlearn, conservative (actual should be higher)
    ground_truth = y_true = xtrain.get_label()
    fpr,tpr = my_roc(ground_truth, predictions)
#     plt.scatter(fpr, tpr)
#     plt.show()
    tpr1 = tpr[(fpr>=0.001).argmax()-1]
    tpr2 = tpr[(fpr>=0.005).argmax()-1] 
    tpr3 = tpr[(fpr>=0.01).argmax()-1]
    return 'score', 0.4 * tpr1 + 0.3 * tpr2 + 0.3* tpr3

def norm_standardize(df, start=0):
    for col in df.columns[start:]:
#         avg = df[col].mean()
#         std = df[col].std(ddof=0)
#         if std != 0:
#             df[col] = (df[col]-avg)/std
#         else:
#             print(col)
        a = df[col]
        z = a
        z[~np.isnan(a)] = zscore(a[~np.isnan(a)])
        df[col] = z
            
def norm_maxmin(df, start=0):
    for col in df.columns[start:]:
        df[col]=(df[col]-df[col].min())/(df[col].max()-df[col].min())

In [14]:
test['f3'].values.shape

(491668,)

In [3]:
dtype = load_obj('dict_dtype')

my_dict = load_obj('my_dict')

In [9]:
data = pd.read_csv("atec_anti_fraud_train.csv",parse_dates=['date'], dtype = dtype)

test = pd.read_csv("atec_anti_fraud_test_a.csv",parse_dates=['date'], dtype = dtype)

In [ ]:
# normalization with maxmin
norm_maxmin(data, 3)
norm_maxmin(test, 2)

In [16]:
# normalization with zscore
norm_standardize(data, 3)
norm_standardize(test, 2)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [18]:
# temporially ignore the rows without labels
data = data[data['label']!=-1]

data.sort_values('date',inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [19]:
df_missing_ratio = load_obj('df_missing_ratio')
# display_all(df_missing_ratio)

In [20]:
selected_cols = ['f'+str(item) for item in df_missing_ratio[df_missing_ratio['positive_missing_ratio']<0.1].index.tolist()]
all_nan_cols = ['f'+str(item) for item in df_missing_ratio[df_missing_ratio['positive_missing_ratio']>0.9].index.tolist()]

In [21]:
selected_cols = []
for col in test.columns[2:]:
    if col not in all_nan_cols:
        selected_cols.append(col)

In [22]:
# use the columns with no or few missing values
data = data[['label']+selected_cols]
test = test[selected_cols]

In [23]:
train = data.iloc[:len(data) * 9 // 10]
valid = data.iloc[len(data) * 9 // 10:]

In [24]:
train_y = train[['label']]
train_x = train.iloc[:,1:]
valid_y = valid[['label']]
valid_x = valid.iloc[:,1:]

In [25]:
xgtrain = xgb.DMatrix(train_x.values, label=train_y.values)
xgval_1 = xgb.DMatrix(valid_x.iloc[:valid_x.shape[0] // 2,:].values,\
                      label=valid_y.iloc[:valid_x.shape[0] // 2,:].values)
xgval_2 = xgb.DMatrix(valid_x.iloc[valid_x.shape[0] // 2:,:].values,\
                      label=valid_y.iloc[valid_x.shape[0] // 2:,:].values)
evallist = [(xgtrain, 'train'), (xgval_1, 'val_1'), (xgval_2, 'val_2')]

In [31]:
# set up the parameters
params = {'max_depth': 8, 'eta': 0.1, 'silent': 1, 'objective': 'binary:logistic'}
params['nthread'] = 4
params['eval_metric'] = ['auc', "error"]
params["scale_pos_weight"] = 5
num_rounds = 20
early_stopping_rounds = 1000

# set up the random seed for testing
params["seed"] = 10

In [35]:
%time xgb_model = xgb.train(params, xgtrain, num_rounds, evallist, feval=my_score3, early_stopping_rounds=early_stopping_rounds, xgb_model = xgb_model)

[0]	train-auc:0.995502	train-error:0.00889	val_1-auc:0.978302	val_1-error:0.014182	val_2-auc:0.978655	val_2-error:0.011616	train-score:0.694961	val_1-score:0.437969	val_2-score:0.508499
Multiple eval metrics have been passed: 'val_2-score' will be used for early stopping.

Will train until val_2-score hasn't improved in 1000 rounds.
[1]	train-auc:0.995536	train-error:0.008856	val_1-auc:0.978359	val_1-error:0.014141	val_2-auc:0.978742	val_2-error:0.011616	train-score:0.696186	val_1-score:0.437344	val_2-score:0.50864
[2]	train-auc:0.995551	train-error:0.008845	val_1-auc:0.97836	val_1-error:0.014121	val_2-auc:0.978695	val_2-error:0.011616	train-score:0.696761	val_1-score:0.437812	val_2-score:0.50864
[3]	train-auc:0.995556	train-error:0.008846	val_1-auc:0.978383	val_1-error:0.014121	val_2-auc:0.97869	val_2-error:0.011636	train-score:0.696993	val_1-score:0.438594	val_2-score:0.50864
[4]	train-auc:0.995588	train-error:0.008832	val_1-auc:0.978316	val_1-error:0.014101	val_2-auc:0.978609	val_2-

In [36]:
# save the model
xgb_model.save_model('model_log/0010.model')
# dump model with feature map
xgb_model.dump_model('model_log/dumpraw0010.txt')

In [37]:
# load the model
xgb_model = xgb.Booster({'nthread': 4})  # init model
xgb_model.load_model('model_log/0010.model')  # load model

In [38]:
# load test data
test_whole = pd.read_feather("tmp/test_native")

In [39]:
xgtest = xgb.DMatrix(test.iloc[:,:].values)

# make predictions
preds = xgb_model.predict(xgtest)#, ntree_limit=xgb_model.best_ntree_limit)

res = pd.concat([test_whole.id, pd.Series(list(preds), name='score')], axis=1)

res.to_csv("0010.csv", index=False)